In [1]:
import numpy as np
import pandas as pd
import glob, os 
import datetime

## Initial data import
Use data starting in 2000

In [32]:
pathname = "./tennis_atp/atp_matches_20??.csv"
df = pd.concat((pd.read_csv(f) for f in glob.iglob(pathname, recursive=True)), ignore_index=True)
# Filter matches without data
df = df[list(map(lambda x: not np.isnan(x), df.loc[:]["minutes"]))]#.head(3000)

df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2004-360,Casablanca,Clay,32.0,A,2004-05-17,1.0,103103.0,1.0,NaN,...,6.0,1.0,4.0,63.0,28.0,17.0,15.0,8.0,5.0,9.0
1,2004-360,Casablanca,Clay,32.0,A,2004-05-17,2.0,102231.0,NaN,NaN,...,11.0,5.0,7.0,97.0,59.0,35.0,14.0,15.0,6.0,13.0
2,2004-360,Casablanca,Clay,32.0,A,2004-05-17,3.0,103700.0,NaN,NaN,...,3.0,0.0,5.0,52.0,23.0,15.0,7.0,8.0,3.0,8.0
3,2004-360,Casablanca,Clay,32.0,A,2004-05-17,4.0,103169.0,8.0,NaN,...,4.0,4.0,6.0,87.0,36.0,25.0,23.0,10.0,17.0,20.0
4,2004-360,Casablanca,Clay,32.0,A,2004-05-17,5.0,103898.0,NaN,NaN,...,7.0,2.0,6.0,68.0,33.0,20.0,16.0,9.0,4.0,8.0


In [33]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

## Match only

In [34]:
games = df[['tourney_id', 'winner_id', 'winner_rank', 'loser_id', 'loser_rank', 'tourney_date', 'match_num']].copy()
def label_data(winner_id, winner_rank, loser_id, loser_rank):
    if winner_rank < loser_rank:
        top_player_rank = winner_rank
        top_player_id = winner_id
        low_player_rank = loser_rank
        low_player_id = loser_id
        top_player_won = 1.
    else:
        top_player_rank = loser_rank
        top_player_id = loser_id
        low_player_rank = winner_rank
        low_player_id = winner_id
        top_player_won = 0.
    return top_player_id, top_player_rank, low_player_id, low_player_rank, top_player_won

games['top_player_id'], games['top_player_rank'], games['low_player_id'], games['low_player_rank'], games['top_player_won'] = \
np.vectorize(label_data)(games['winner_id'], games['winner_rank'], games['loser_id'], games['loser_rank'])
games.drop(['winner_id', 'winner_rank', 'loser_id', 'loser_rank'], axis=1, inplace=True)
games.head()
        

,tourney_id,tourney_date,match_num,top_player_id,top_player_rank,low_player_id,low_player_rank,top_player_won
0,2004-360,2004-05-17,1.0,103103.0,32.0,104259.0,149.0,1.0
1,2004-360,2004-05-17,2.0,103153.0,101.0,102231.0,150.0,0.0
2,2004-360,2004-05-17,3.0,103700.0,136.0,102558.0,165.0,1.0
3,2004-360,2004-05-17,4.0,103169.0,90.0,103007.0,451.0,1.0
4,2004-360,2004-05-17,5.0,103105.0,80.0,103898.0,109.0,0.0


### Database by player

In [35]:
winners = (df[list(df.columns[:7]) + [column for column in df.columns if column[0]=='w'] + ['l_bpFaced', 'l_bpSaved']]
               .rename(columns={'l_bpFaced': '_break_points_for', 'l_bpSaved':'_break_points_missed'})
          )

for column in winners.columns[7:]:
    index = column.find('_') + 1
    winners.rename(columns={column: column[index:]}, inplace = True)
    
losers = (df[list(df.columns[:7]) + [column for column in df.columns if column[0]=='l'] + ['w_bpFaced', 'w_bpSaved']]
               .rename(columns={'w_bpFaced': '_break_points_for', 'w_bpSaved':'_break_points_missed'})
          )
for column in losers.columns[7:]:
    index = column.find('_') + 1
    losers.rename(columns={column: column[index:]}, inplace = True)

players = pd.DataFrame.append(winners, losers)
players['id'] = players['id'].astype('int')

players['%1st_serve_in'] = players['1stIn']/players['svpt']
players['%1st_serve_won'] = players['1stWon']/players['1stIn']
players['%2nd_serve_won'] = players['2ndWon']/(players['svpt'] - players['1stIn'])
players['%break_points_saved'] = players['bpSaved']/players['bpFaced']
players['%break_points_converted'] = 1. - players['break_points_missed']/players['break_points_for']

players.rename(columns={
    'df': 'double_fault',
    'bpFaced': 'break_points_against'
}, inplace=True)

#players.set_index(['id','tourney_date'], inplace=True)

players.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,SvGms,bpSaved,break_points_against,break_points_for,break_points_missed,%1st_serve_in,%1st_serve_won,%2nd_serve_won,%break_points_saved,%break_points_converted
0,2004-360,Casablanca,Clay,32.0,A,2004-05-17,1.0,103103,1.0,NaN,...,8.0,6.0,6.0,9.0,5.0,0.571429,0.843750,0.458333,1.000000,0.444444
1,2004-360,Casablanca,Clay,32.0,A,2004-05-17,2.0,102231,NaN,NaN,...,14.0,6.0,11.0,13.0,6.0,0.710843,0.559322,0.500000,0.545455,0.538462
2,2004-360,Casablanca,Clay,32.0,A,2004-05-17,3.0,103700,NaN,NaN,...,7.0,3.0,3.0,8.0,3.0,0.520000,0.769231,0.541667,1.000000,0.625000
3,2004-360,Casablanca,Clay,32.0,A,2004-05-17,4.0,103169,8.0,NaN,...,10.0,3.0,4.0,20.0,17.0,0.650000,0.692308,0.619048,0.750000,0.150000
4,2004-360,Casablanca,Clay,32.0,A,2004-05-17,5.0,103898,NaN,NaN,...,9.0,6.0,7.0,8.0,4.0,0.589041,0.674419,0.533333,0.857143,0.500000


## Statistics

In [36]:
stats = ['ace', 'double_fault', '%1st_serve_in', '%1st_serve_won', '%2nd_serve_won', '%break_points_saved', 'break_points_against', '%break_points_converted']

### Life to date

In [37]:
def life_to_date_stats(df, player_id, tdate, match_num):
    tdate = pd.to_datetime(tdate) # Issues with format in vectorize
    column_names = [stat + '_ltd' for stat in stats]
    results = (df.loc[(df['id'] == player_id) & (df['tourney_date'] < tdate) & (df['match_num'] < match_num)]
                    .groupby('id')
                    .agg('mean')[stats]
                    .rename(columns=dict(zip(stats, column_names)))
              )
    results['tourney_date'] = tdate
    results['match_num'] = match_num
    return results
    
ltd_stats = pd.concat(np.vectorize(life_to_date_stats, excluded=['df'])(df=players, player_id=players['id'], tdate=players['tourney_date'], match_num=players['match_num']))
players = players.merge(ltd_stats, how='left', on=['id', 'tourney_date', 'match_num'])


### Last x games

In [38]:
def last_x_games_avg(df, player_id, tdate, match_num, x):
    tdate = pd.to_datetime(tdate) # Issues with format in vectorize
    column_names = [stat + '_last_' + str(x) + '_games' for stat in stats]
    results = (df.loc[(df['id'] == player_id) & 
                      (df['tourney_date'] < tdate) & 
                      (df['match_num'] < match_num)]
                 .sort_values(by=['tourney_date', 'match_num'], ascending=False)
                 .head(x)
                 .groupby('id')
                 .agg('mean')[stats]
                 .rename(columns=dict(zip(stats, column_names)))
              )
    results['tourney_date'] = tdate
    results['match_num'] = match_num
    return results

def merge_x_games_avg(df, x):
    last_x_games = pd.concat(np.vectorize(last_x_games_avg, excluded=['df', 'x'])(df=df, player_id=df['id'], tdate=df['tourney_date'], match_num=df['match_num'], x=x))
    df = df.merge(last_x_games, how='left', on=['id', 'tourney_date', 'match_num'])
    return df

players = merge_x_games_avg(players, 10)

### Current year

In [ ]:
def current_year_stats(df, player_id, tdate, match_num):
    tdate = pd.to_datetime(tdate) # Issues with format in vectorize
    column_names = [stat + '_current_year' for stat in stats]
    results = (df.loc[(df['id'] == player_id) & 
                      (df['tourney_date'] < tdate) & 
                      (df['match_num'] < match_num) & 
                      (df['tourney_date'].map(lambda x: x.year) < tdate.year)]
                    .groupby('id')
                    .agg('mean')[stats]
                    .rename(columns=dict(zip(stats, column_names)))
              )
    results['tourney_date'] = tdate
    results['match_num'] = match_num
    return results
    
ltd_stats = pd.concat(np.vectorize(life_to_date_stats, excluded=['df'])(df=players, player_id=players['id'], tdate=players['tourney_date'], match_num=players['match_num']))
players = players.merge(ltd_stats, how='left', on=['id', 'tourney_date', 'match_num'])

### Face to face

In [ ]:
def face_to_face(games_df, players_df, top_player_id,  low_player_id, tdate, match_num):
    tdate = pd.to_datetime(tdate) # Issues with format in vectorize
    top_stats = [stat + '_top_player_f2f' for stat in stats]
    low_stats = [stat + '_low_player_f2f' for stat in stats]
    results = (games_df.loc[(games['top_player_id'].isin([top_player_id, low_player_id])) & 
                          (games['low_player_id'].isin([top_player_id, low_player_id])) & 
                          (games['tourney_date'] < tdate) & 
                          (games['match_num'] < match_num)]
                     .merge(players.rename(columns=dict(zip(stats, top_stats))), 
                            how='left', 
                            left_on=['top_player_id', 'tourney_date', 'match_num'],
                            right_on=['id', 'tourney_date', 'match_num'])
                     .merge(players.rename(columns=dict(zip(stats, low_stats))), 
                            how='left', 
                            left_on=['low_player_id', 'tourney_date', 'match_num'],
                            right_on=['id', 'tourney_date', 'match_num'])
                     .groupby(['top_player_id', 'low_player_id'])
                     .agg('mean')[top_stats + low_stats + ['top_player_won']]
            )
    results['tourney_date'] = tdate
    results['match_num'] = match_num
    return results

games = pd.concat(
    np.vectorize(face_to_face, excluded=['games_df', 'players_df'])(
        games_df = games, players_df=players, top_player_id=games['top_player_id'], low_player_id=games['low_player_id'], tdate=games['tourney_date'], match_num=games['match_num']
    )
)
games.head()

### Complete BD

In [ ]:
games = (games.merge(players.rename(columns=dict(zip(stats, top_stats))), 
                     how='left', 
                     left_on=['top_player_id', 'tourney_date', 'match_num'],
                     right_on=['id', 'tourney_date', 'match_num'])
              .merge(players.rename(columns=dict(zip(stats, low_stats))), 
                     how='left', 
                     left_on=['low_player_id', 'tourney_date', 'match_num'],
                     right_on=['id', 'tourney_date', 'match_num'])